In [107]:
#Importação de pacotes
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import joblib # Utilizado para salvar o modelo preditivo
from sklearn.preprocessing import LabelEncoder #Utilizada para fazer o OneHotEncoding
from sklearn.metrics import mean_squared_error,precision_score, recall_score, f1_score, accuracy_score, roc_auc_score, confusion_matrix
from imblearn import under_sampling, over_sampling #Utilizada para fazer o balanceamento de dados
from imblearn.over_sampling import SMOTE #Utilizada para fazer o balanceamento de dados
from sklearn.preprocessing import MinMaxScaler #Utilizada para fazer a padronização dos dados
from sklearn.metrics import r2_score # Utilizado para medir a acuracia do modelo preditivo
import pymssql as sql #conexão SQL

warnings.filterwarnings("ignore") 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [108]:

#criando a conexao com o db
conexao = sql.connect('DESKTOP-NV3ARNG', 'usuario_python', '123456', 'MODELOS_PREDITIVOS')
# criando o df com a consulta na tabela EXTRACAO_DADOS_SISTEMA
df = pd.read_sql_query('select * from EXTRACAO_DADOS_SISTEMA', conexao)
# fechando conexão com db
conexao.close()


In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10415 entries, 0 to 10414
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   NUMERO_CONTRATO                 10415 non-null  int64  
 1   DATA_ASSINATURA_CONTRATO        10415 non-null  object 
 2   TIPO_FINANCIAMENTO              10415 non-null  object 
 3   TAXA_AO_ANO                     10415 non-null  float64
 4   PZ_FINANCIAMENTO                10415 non-null  int64  
 5   CIDADE_CLIENTE                  10415 non-null  object 
 6   ESTADO_CLIENTE                  10415 non-null  object 
 7   RENDA_MENSAL_CLIENTE            10415 non-null  float64
 8   QT_PC_ATRASO                    10415 non-null  int64  
 9   QT_DIAS_PRIM_PC_ATRASO          10415 non-null  int64  
 10  QT_TOTAL_PC_PAGAS               10415 non-null  int64  
 11  VL_TOTAL_PC_PAGAS               10411 non-null  float64
 12  QT_PC_PAGA_EM_DIA               

In [110]:
df.isna().sum()
#Verifica-se valores 5 valores nulos
#4 na var vl_total_pc_pagas e 1 na var idade_assinatura_contrato


NUMERO_CONTRATO                   0
DATA_ASSINATURA_CONTRATO          0
TIPO_FINANCIAMENTO                0
TAXA_AO_ANO                       0
PZ_FINANCIAMENTO                  0
CIDADE_CLIENTE                    0
ESTADO_CLIENTE                    0
RENDA_MENSAL_CLIENTE              0
QT_PC_ATRASO                      0
QT_DIAS_PRIM_PC_ATRASO            0
QT_TOTAL_PC_PAGAS                 0
VL_TOTAL_PC_PAGAS                 4
QT_PC_PAGA_EM_DIA                 0
QT_DIAS_MIN_ATRASO                0
QT_DIAS_MAX_ATRASO                0
QT_DIAS_MEDIA_ATRASO              0
VALOR_FINANCIAMENTO               0
VALOR_PARCELA                     0
IDADE_DATA_ASSINATURA_CONTRATO    1
INADIMPLENTE_COBRANCA             0
dtype: int64

In [111]:
#Verificando se com os dados de outras colunas consigo tratar os valores nulos
nulas = df[df['VL_TOTAL_PC_PAGAS'].isnull() + df['IDADE_DATA_ASSINATURA_CONTRATO'].isnull()]
nulas.head()

,NUMERO_CONTRATO,DATA_ASSINATURA_CONTRATO,TIPO_FINANCIAMENTO,TAXA_AO_ANO,PZ_FINANCIAMENTO,CIDADE_CLIENTE,ESTADO_CLIENTE,RENDA_MENSAL_CLIENTE,QT_PC_ATRASO,QT_DIAS_PRIM_PC_ATRASO,QT_TOTAL_PC_PAGAS,VL_TOTAL_PC_PAGAS,QT_PC_PAGA_EM_DIA,QT_DIAS_MIN_ATRASO,QT_DIAS_MAX_ATRASO,QT_DIAS_MEDIA_ATRASO,VALOR_FINANCIAMENTO,VALOR_PARCELA,IDADE_DATA_ASSINATURA_CONTRATO,INADIMPLENTE_COBRANCA
5792,132882,2016-08-23,IMOBILIARIO,9.0,100,SAO PAULO,SP,1800.0,0,0,35,23064.46,30,23,87,44,455000.0,4608.50,NaN,NAO
7517,53380,2017-10-24,IMOBILIARIO,22.0,220,VALPARAISO DE GOIAS,GO,5800.0,62,1837,0,NaN,0,0,0,0,200000.0,1109.09,49.0,SIM
7718,72587,2014-08-11,IMOBILIARIO,18.0,240,BRASILIA,DF,7800.0,100,3006,0,NaN,0,0,0,0,350000.0,1720.83,24.0,SIM
8399,74795,2013-05-10,IMOBILIARIO,19.0,240,BRASILIA,DF,7800.0,114,3464,0,NaN,0,0,0,0,550000.0,2727.08,43.0,SIM
10197,79980,2017-02-21,IMOBILIARIO,17.0,72,BRASILIA,DF,7800.0,69,2082,0,NaN,0,0,0,0,280000.0,3983.33,53.0,NAO


In [112]:
df = df.dropna()
df.isna().sum()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10410 entries, 0 to 10414
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   NUMERO_CONTRATO                 10410 non-null  int64  
 1   DATA_ASSINATURA_CONTRATO        10410 non-null  object 
 2   TIPO_FINANCIAMENTO              10410 non-null  object 
 3   TAXA_AO_ANO                     10410 non-null  float64
 4   PZ_FINANCIAMENTO                10410 non-null  int64  
 5   CIDADE_CLIENTE                  10410 non-null  object 
 6   ESTADO_CLIENTE                  10410 non-null  object 
 7   RENDA_MENSAL_CLIENTE            10410 non-null  float64
 8   QT_PC_ATRASO                    10410 non-null  int64  
 9   QT_DIAS_PRIM_PC_ATRASO          10410 non-null  int64  
 10  QT_TOTAL_PC_PAGAS               10410 non-null  int64  
 11  VL_TOTAL_PC_PAGAS               10410 non-null  float64
 12  QT_PC_PAGA_EM_DIA               10410

In [113]:
#Verificar valores unicos
valores_unicos = []
for i in df.columns[0:20].tolist():
    print(i, ':', len(df[i].astype(str).value_counts()))
    valores_unicos.append(len(df[i].astype(str).value_counts()))
#Verifica-se que o tipo de financiamento tem só um tipo de valor então não vai ajudar na análise vou escluir a coluna

NUMERO_CONTRATO : 10410
DATA_ASSINATURA_CONTRATO : 1069
TIPO_FINANCIAMENTO : 1


TAXA_AO_ANO : 111
PZ_FINANCIAMENTO : 9
CIDADE_CLIENTE : 525
ESTADO_CLIENTE : 27
RENDA_MENSAL_CLIENTE : 7
QT_PC_ATRASO : 130
QT_DIAS_PRIM_PC_ATRASO : 227
QT_TOTAL_PC_PAGAS : 136
VL_TOTAL_PC_PAGAS : 9859
QT_PC_PAGA_EM_DIA : 129
QT_DIAS_MIN_ATRASO : 122
QT_DIAS_MAX_ATRASO : 534
QT_DIAS_MEDIA_ATRASO : 323
VALOR_FINANCIAMENTO : 834
VALOR_PARCELA : 1826
IDADE_DATA_ASSINATURA_CONTRATO : 78
INADIMPLENTE_COBRANCA : 2


In [114]:
# Verificar se a variavel target precisará ser balanceada
df.groupby(['INADIMPLENTE_COBRANCA']).size()
#Precisa de balanceamento

INADIMPLENTE_COBRANCA
NAO    2375
SIM    8035
dtype: int64

In [115]:
# Criando faixa de prazos para utilizarmos no modelo preditivo
bins = [-100, 120, 180, 240]
labels = ['Até 120 Meses', '121 até 180 Meses', '181 até 240 Meses']
df['FAIXA_PRAZO_FINANCIAMENTO'] = pd.cut(df['PZ_FINANCIAMENTO'], bins=bins, labels=labels)
pd.value_counts(df.FAIXA_PRAZO_FINANCIAMENTO)

FAIXA_PRAZO_FINANCIAMENTO
Até 120 Meses        7402
181 até 240 Meses    2241
121 até 180 Meses     767
Name: count, dtype: int64

In [116]:
# Criando faixa salarial para utilizarmos no modelo preditivo
bins = [-100, 100000, 200000, 300000, 400000, 500000, 750000, 1000000, 9000000000]
labels = ['Até 100 mil', '101 até 200 mil', '201 até 300 mil', '301 até 400 mil', '401 até 500 mil', 
          '501 até 750 mil', 'De 751 até 1.000.000','Mais de 1.000.000']
df['FAIXA_VALOR_FINANCIADO'] = pd.cut(df['VALOR_FINANCIAMENTO'], bins=bins, labels=labels)
pd.value_counts(df.FAIXA_VALOR_FINANCIADO)

FAIXA_VALOR_FINANCIADO
201 até 300 mil         3949
301 até 400 mil         2098
101 até 200 mil         1823
401 até 500 mil         1128
501 até 750 mil          915
De 751 até 1.000.000     236
Até 100 mil              167
Mais de 1.000.000         94
Name: count, dtype: int64

In [117]:
# Vou retirar a variável TIPO FINANCIAMENTO pois só existe 1 tipo
# Vou retirar a variável VALOR_FINANCIAMENTO, criei uma variavel de faixa de valores para ela.
# Vou retirar a variável PRAZO_FINANCIAMENTO, criei uma variavel de faixa de meses para ela.
# Vou retirar a variável DATA_ASSINATURA_CONTRATO não interfere no modelo
# Vou retirar a variável NUMERO_CONTRATO não interfere no modelo

colunas = ['TAXA_AO_ANO', 'CIDADE_CLIENTE', 'ESTADO_CLIENTE','RENDA_MENSAL_CLIENTE', 
           'QT_PC_ATRASO', 'QT_DIAS_PRIM_PC_ATRASO','QT_TOTAL_PC_PAGAS',
           'VL_TOTAL_PC_PAGAS', 'QT_PC_PAGA_EM_DIA','QT_DIAS_MIN_ATRASO',
           'QT_DIAS_MAX_ATRASO', 'QT_DIAS_MEDIA_ATRASO','VALOR_PARCELA',
           'IDADE_DATA_ASSINATURA_CONTRATO', 'FAIXA_VALOR_FINANCIADO',
           'FAIXA_PRAZO_FINANCIAMENTO','INADIMPLENTE_COBRANCA']

df_dados = pd.DataFrame(df, columns=colunas)

In [118]:
df_dados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10410 entries, 0 to 10414
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   TAXA_AO_ANO                     10410 non-null  float64 
 1   CIDADE_CLIENTE                  10410 non-null  object  
 2   ESTADO_CLIENTE                  10410 non-null  object  
 3   RENDA_MENSAL_CLIENTE            10410 non-null  float64 
 4   QT_PC_ATRASO                    10410 non-null  int64   
 5   QT_DIAS_PRIM_PC_ATRASO          10410 non-null  int64   
 6   QT_TOTAL_PC_PAGAS               10410 non-null  int64   
 7   VL_TOTAL_PC_PAGAS               10410 non-null  float64 
 8   QT_PC_PAGA_EM_DIA               10410 non-null  int64   
 9   QT_DIAS_MIN_ATRASO              10410 non-null  int64   
 10  QT_DIAS_MAX_ATRASO              10410 non-null  int64   
 11  QT_DIAS_MEDIA_ATRASO            10410 non-null  int64   
 12  VALOR_PARCELA          

In [119]:
df_dados.head()

,TAXA_AO_ANO,CIDADE_CLIENTE,ESTADO_CLIENTE,RENDA_MENSAL_CLIENTE,QT_PC_ATRASO,QT_DIAS_PRIM_PC_ATRASO,QT_TOTAL_PC_PAGAS,VL_TOTAL_PC_PAGAS,QT_PC_PAGA_EM_DIA,QT_DIAS_MIN_ATRASO,QT_DIAS_MAX_ATRASO,QT_DIAS_MEDIA_ATRASO,VALOR_PARCELA,IDADE_DATA_ASSINATURA_CONTRATO,FAIXA_VALOR_FINANCIADO,FAIXA_PRAZO_FINANCIAMENTO,INADIMPLENTE_COBRANCA
0,15.0,FORTALEZA,CE,4800.0,0,0,97,120078.00,72,3,506,152,471.29,70.0,Até 100 mil,121 até 180 Meses,SIM
1,15.0,SAO PAULO,SP,6800.0,0,0,103,156759.47,42,1,349,93,1858.69,28.0,301 até 400 mil,181 até 240 Meses,SIM
2,18.0,SAO PAULO,SP,6800.0,0,0,98,277125.05,83,1,151,39,3123.57,60.0,501 até 750 mil,181 até 240 Meses,SIM
3,15.0,SAO PAULO,SP,6800.0,0,0,87,351395.19,66,1,3797,206,2873.58,41.0,401 até 500 mil,181 até 240 Meses,SIM
4,17.0,SAO PAULO,SP,6800.0,0,0,71,344249.83,57,1,36,10,4224.03,63.0,501 até 750 mil,181 até 240 Meses,SIM


## Analise exploratoria das variaveis categoricas

In [120]:
import plotly.express as px
# Contar os valores para cada inadimplente
value_counts = df_dados['INADIMPLENTE_COBRANCA'].value_counts().reset_index()
value_counts.columns = ['inadimplente', 'quantidade']

# Criar o gráfico de barras
fig = px.bar(value_counts, 
             x='inadimplente', 
             y='quantidade', 
             title='quantidade de inadimplentes', 
             color='inadimplente',
             color_discrete_sequence=px.colors.qualitative.Antique,
             text='quantidade')  # Adiciona o texto nas barras
fig.update_layout(
    xaxis_title='É INADIMPLENTE?',
    yaxis_title='Quantidade',
    legend_title='Inadimplente'
)


fig.show()

In [121]:
# Contar os valores para cada faixa de valor financiado e inadimplente
value_counts1 = df_dados.groupby([ 'FAIXA_VALOR_FINANCIADO','INADIMPLENTE_COBRANCA']).size().reset_index(name='quantidade')
value_counts1.columns = ['faixa de valor financiado', 'inadimplente', 'quantidade']
# Criar o gráfico de barras
fig = px.bar(value_counts1, 
             x='faixa de valor financiado', 
             y='quantidade', 
             color='inadimplente',
             title='Faixa de Valor Financiado por Inadimplente',
             color_discrete_sequence=px.colors.qualitative.Antique,
             text='quantidade')  # Adiciona o texto nas barras

# Ajustar a rotação dos rótulos do eixo x e outros ajustes de layout
fig.update_layout(
    xaxis_title='Faixa de Valor Financiado',
    yaxis_title='Quantidade',
    legend_title='Inadimplente'
)


# Mostrar o gráfico
fig.show()

In [122]:
# Contar os valores para cada faixa de valor financiado e inadimplente
value_counts1 = df_dados.groupby([ 'FAIXA_VALOR_FINANCIADO','INADIMPLENTE_COBRANCA']).size().reset_index(name='quantidade')
value_counts1.columns = ['faixa de valor financiado', 'inadimplente', 'quantidade']
# Criar o gráfico de barras
fig = px.bar(value_counts1, 
             x='faixa de valor financiado', 
             y='quantidade', 
             color='inadimplente',
             title='Faixa de Valor Financiado por Inadimplente',
             color_discrete_sequence=px.colors.qualitative.Antique,
             text='quantidade')  # Adiciona o texto nas barras

# Ajustar a rotação dos rótulos do eixo x e outros ajustes de layout
fig.update_layout(
    xaxis_title='Faixa de Valor Financiado',
    yaxis_title='Quantidade',
    legend_title='Inadimplente'
)


# Mostrar o gráfico
fig.show()

In [123]:
# Contar os valores para cada faixa de valor financiado e inadimplente
value_counts1 = df_dados.groupby([ 'FAIXA_PRAZO_FINANCIAMENTO','INADIMPLENTE_COBRANCA']).size().reset_index(name='quantidade')
value_counts1.columns = ['faixa de prazo financiado', 'inadimplente', 'quantidade']
# Criar o gráfico de barras
fig = px.bar(value_counts1, 
             x='faixa de prazo financiado', 
             y='quantidade', 
             color='inadimplente',
             title='Faixa de Prazo Financiado por Inadimplente',
             color_discrete_sequence=px.colors.qualitative.Antique,
             text='quantidade')  # Adiciona o texto nas barras

# Ajustar a rotação dos rótulos do eixo x e outros ajustes de layout
fig.update_layout(
    xaxis_title='Faixa de Prazo Financiado',
    yaxis_title='Quantidade',
    legend_title='Inadimplente'
)


# Mostrar o gráfico
fig.show()

# Analise das variáveis numéricas

In [124]:
#carregar variaveis para plot
#Entender as variaveis e identificar se necessita de tratamento de outilier
variaveis_numericas = []
for i in df_dados.columns[0:17].tolist():
        if df_dados.dtypes[i] == 'int64' or df_dados.dtypes[i] == 'float64':                        
            variaveis_numericas.append(i)   

variaveis_numericas

['TAXA_AO_ANO',
 'RENDA_MENSAL_CLIENTE',
 'QT_PC_ATRASO',
 'QT_DIAS_PRIM_PC_ATRASO',
 'QT_TOTAL_PC_PAGAS',
 'VL_TOTAL_PC_PAGAS',
 'QT_PC_PAGA_EM_DIA',
 'QT_DIAS_MIN_ATRASO',
 'QT_DIAS_MAX_ATRASO',
 'QT_DIAS_MEDIA_ATRASO',
 'VALOR_PARCELA',
 'IDADE_DATA_ASSINATURA_CONTRATO']

In [125]:
import plotly.subplots as sp

# Configurando o layout do subplot
fig = sp.make_subplots(rows=4, cols=3, subplot_titles=variaveis_numericas)

# Adicionando os boxplots para cada variável
for idx, var in enumerate(variaveis_numericas):
    row = idx // 3 + 1  # Linha do subplot
    col = idx % 3 + 1   # Coluna do subplot
    
    # Criando o boxplot para a variável atual
    fig.add_trace(
        px.box(df_dados, y=var).data[0],
        row=row,
        col=col
    )

# Ajustando o layout do gráfico
fig.update_layout(
    height=1400,
    width=1400,
    showlegend=False,
    title_text="Boxplots das Variáveis Numéricas"
)

# Exibindo o gráfico
fig.show()

In [126]:
# Colocamos o SLICE até 16 porque NÃO precisamos fazer OneHotEncoding para variavel TARGET 
variaveis_categoricas = []
for i in df_dados.columns[0:16].tolist():
        if df_dados.dtypes[i] == 'object' or df_dados.dtypes[i] == 'category':                        
            variaveis_categoricas.append(i)

variaveis_categoricas    

['CIDADE_CLIENTE',
 'ESTADO_CLIENTE',
 'FAIXA_VALOR_FINANCIADO',
 'FAIXA_PRAZO_FINANCIAMENTO']

In [127]:
# Cria o encoder e aplica OneHotEncoder
lb = LabelEncoder()

for var in variaveis_categoricas:
    df_dados[var] = lb.fit_transform(df_dados[var])

df_dados.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10410 entries, 0 to 10414
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   TAXA_AO_ANO                     10410 non-null  float64
 1   CIDADE_CLIENTE                  10410 non-null  int32  
 2   ESTADO_CLIENTE                  10410 non-null  int32  
 3   RENDA_MENSAL_CLIENTE            10410 non-null  float64
 4   QT_PC_ATRASO                    10410 non-null  int64  
 5   QT_DIAS_PRIM_PC_ATRASO          10410 non-null  int64  
 6   QT_TOTAL_PC_PAGAS               10410 non-null  int64  
 7   VL_TOTAL_PC_PAGAS               10410 non-null  float64
 8   QT_PC_PAGA_EM_DIA               10410 non-null  int64  
 9   QT_DIAS_MIN_ATRASO              10410 non-null  int64  
 10  QT_DIAS_MAX_ATRASO              10410 non-null  int64  
 11  QT_DIAS_MEDIA_ATRASO            10410 non-null  int64  
 12  VALOR_PARCELA                   10410

In [128]:
#Separar variaveis preditoras e target
PREDITORAS = df_dados.iloc[:, 0:15]  
TARGET = df_dados.iloc[:, 16] 
PREDITORAS.head()


,TAXA_AO_ANO,CIDADE_CLIENTE,ESTADO_CLIENTE,RENDA_MENSAL_CLIENTE,QT_PC_ATRASO,QT_DIAS_PRIM_PC_ATRASO,QT_TOTAL_PC_PAGAS,VL_TOTAL_PC_PAGAS,QT_PC_PAGA_EM_DIA,QT_DIAS_MIN_ATRASO,QT_DIAS_MAX_ATRASO,QT_DIAS_MEDIA_ATRASO,VALOR_PARCELA,IDADE_DATA_ASSINATURA_CONTRATO,FAIXA_VALOR_FINANCIADO
0,15.0,152,5,4800.0,0,0,97,120078.00,72,3,506,152,471.29,70.0,5
1,15.0,453,25,6800.0,0,0,103,156759.47,42,1,349,93,1858.69,28.0,2
2,18.0,453,25,6800.0,0,0,98,277125.05,83,1,151,39,3123.57,60.0,4
3,15.0,453,25,6800.0,0,0,87,351395.19,66,1,3797,206,2873.58,41.0,3
4,17.0,453,25,6800.0,0,0,71,344249.83,57,1,36,10,4224.03,63.0,4


In [129]:
TARGET.head()

0    SIM
1    SIM
2    SIM
3    SIM
4    SIM
Name: INADIMPLENTE_COBRANCA, dtype: object

In [130]:
# Seed para reproduzir o mesmo resultado
seed = 100

# Cria o balanceador SMOTE
balanceador = SMOTE(random_state = seed)

# Aplica o balanceador
PREDITORAS_RES, TARGET_RES = balanceador.fit_resample(PREDITORAS, TARGET)

In [131]:
# Contar a frequência de cada valor
contagem_target = TARGET_RES.value_counts().reset_index()
contagem_target.columns = ['Categoria', 'Contagem']

# Criar o gráfico de barras
fig = px.bar(contagem_target,
             x='Categoria',
             y='Contagem',
             title='Inadimplentes x Não Inadimplentes',
             color='Categoria',
             color_discrete_map={'Inadimplente': '#FF7F0E', 'Não Inadimplente': '#1F77B4'})

# Exibir o gráfico
fig.show()
#Verifica-se que o balanceamento foi feito